<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/11-integrate-cimg-library/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp69ev3ln4".
Sun Apr  7 17:30:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                 

In [2]:
# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

--2024-04-07 17:30:29--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2024-04-07 17:30:29--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

cpp_plugin.py       100%[===================>]   2.67K  --.-KB/s    in 0s      

2024-04-07 17:30:2

In [ ]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!ls '/content/drive/My Drive/GPU Computing/human detection dataset'

0  1


In [ ]:
!git clone --recursive https://github.com/YoungYoung619/pedestrian-detection-in-hazy-weather.git

In [ ]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>


namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

//__global__ void computeGradients(unsigned char* image, float *d_magnitude, float *d_orientation, float *d_histograms, int width, int height, int cellSize) {
__global__ void computeGradients(unsigned char* image, float *d_histograms, int width, int height, int cellSize) {

    int idx = (blockIdx.x * blockDim.x) + threadIdx.x;
    int idy = (blockIdx.y * blockDim.y) + threadIdx.y;
    //int indexCurrent = idy * width + idx;

    //int blockId = blockIdx.x + blockIdx.y * gridDim.x;
    //int threadId = blockId * (blockDim.x * blockDim.y) + (threadIdx.y * blockDim.x) + threadIdx.x;


    if (idx >= width || idy >= height) return; // Boundary check

    float G_x = 0;
    if (idx > 0 && idx < width - 1) {
        G_x = (float)image[idy * width + (idx + 1)] - (float)image[idy * width + (idx - 1)];
    }

    float G_y = 0;
    if (idy > 0 && idy < height - 1) {
        G_y = (float)image[(idy + 1) * width + idx] - (float)image[(idy - 1) * width + idx];
    }
    float d_magnitude = sqrtf(G_x * G_x + G_y * G_y);
    float d_orientation = atan2f(G_y, G_x);
    //d_magnitude[indexCurrent] = sqrtf(G_x * G_x + G_y * G_y);
    //d_orientation[indexCurrent] = atan2f(G_y, G_x);

    // Compute histogram bin for the current gradient
    int cellX = idx / cellSize;
    int cellY = idy / cellSize;
    int histIndex = cellY * (width / cellSize) + cellX;
    int numBins = 9; // Assuming 9 orientation bins
    float binWidth = M_PI / numBins;
    //int bin = floor((d_orientation[indexCurrent] + M_PI) / binWidth);
    int bin = floor((d_orientation + M_PI) / binWidth);
    if (bin == numBins) bin = 0; // Wrap around
    //atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude[indexCurrent]);
    //atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude);

    /*if((idx == 2 && idy == 2) || (idx==300 & idy == 300)){
      //printf("Value for idx %d e idy %d - bin value: %d - orientation value %f \n", idx, idy, bin,d_orientation[indexCurrent] + M_PI);
    }*/
}

std::vector<float> computeDescriptors(const std::string& image_path){
    // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }



    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);
    cudaError_t status;
    /*size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    status = cudaMalloc((void **)&d_magnitude, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_magnitude, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    sizeInBytes = image.total() * sizeof(float);
    float* d_orientation;
    status = cudaMalloc((void **)&d_orientation, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_orientation, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }*/

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(8, 8);
    dim3 gridSize((image.cols) / blockSize.x,
                  (image.rows) / blockSize.y);


    // Allocate memory for histograms
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    size_t histSize = numCellsX * numCellsY * 9 * sizeof(float);
    float* d_histograms;
    status = cudaMalloc((void **)&d_histograms, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_histograms, 0, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    // Start measuring time
    auto start = std::chrono::high_resolution_clock::now();
    // Launch the kernel
    //computeGradients<<<gridSize, blockSize>>>(d_image, d_magnitude, d_orientation, d_histograms, image.cols, image.rows, cellSize);
    computeGradients<<<gridSize, blockSize>>>(d_image, d_histograms, image.cols, image.rows, cellSize);

    cudaDeviceSynchronize();

    auto end = std::chrono::high_resolution_clock::now();

    // Compute the duration and print it
    std::chrono::duration<double> duration = end - start;
    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;


    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    cudaMemcpy(h_histograms, d_histograms, histSize, cudaMemcpyDeviceToHost);

    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        for (int j = 0; j < 9; ++j) {
            std::cout << "Cell " << i << ", Bin " << j << ": " << h_histograms[i * 9 + j] << std::endl;
        }
    }*/
    // Cleanup

    // Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }
    cudaFree(d_image);
    //cudaFree(d_magnitude);
    //cudaFree(d_orientation);
    delete[] h_histograms;
    cudaFree(d_histograms);

    return descriptor;
}

int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    for (int i=1; i <= 144; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();

     std::string image_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0/116.png";

     std::vector<float> descriptor = computeDescriptors(image_path);
    if (descriptor.empty()) {
        std::cout << "Vector is empty" << std::endl;
    } else {
        int label = 1;
        std::vector<float> descriptor1;
        int length = static_cast<int>(descriptor.size());
        std::cout << "Dimension of descriptor: " << length << std::endl;
        descriptor1.push_back(length);
        saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
        descriptor.clear(); // Clear the vector
        descriptor1.clear();
    }
    //People present class
    /*for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;


        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 1;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }
    }*/

      //Not people present class
      /*folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0";
      for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;


        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 0;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }

    }*/


    return 0;
}


Overwriting gradient_computation.cu


In [ ]:
!nvcc gradient_computation.cu -o gradient_computation -diag-suppress 611 `pkg-config --cflags --libs opencv4` -run


gradient_computation.cu(79): warning #177-D: variable "histIndex" was declared but never referenced
      int histIndex = cellY * (width / cellSize) + cellX;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Execution time: 0.000149876 seconds
Dimension of descriptor: 144


Execution time: 0.00765783 seconds


Execution time: 0.232246 seconds


In [ ]:
!./gradient_computation

Processing image: 1/1.png
Dimension of descriptor: 26244
Processing image: 1/0.png
Dimension of descriptor: 26244
Processing image: 1/3.png
Dimension of descriptor: 26244
Processing image: 1/2.png
Dimension of descriptor: 26244


In [ ]:
!sudo apt-get install g++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
%%writefile hello.cpp
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>


namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

void computeGradientsSequential(cv::Mat& image, float *d_histograms, int cellSize) {
    int width = image.cols;
    int height = image.rows;

    for (int idy = 0; idy < height; ++idy) {
        for (int idx = 0; idx < width; ++idx) {
            if (idx >= width || idy >= height) continue; // Boundary check

            float G_x = 0;
            if (idx > 0 && idx < width - 1) {
                G_x = static_cast<float>(image.at<uchar>(idy, idx + 1)) - static_cast<float>(image.at<uchar>(idy, idx - 1));
            }

            float G_y = 0;
            if (idy > 0 && idy < height - 1) {
                G_y = static_cast<float>(image.at<uchar>(idy + 1, idx)) - static_cast<float>(image.at<uchar>(idy - 1, idx));
            }
            float d_magnitude = sqrtf(G_x * G_x + G_y * G_y);
            float d_orientation = atan2f(G_y, G_x);

            // Compute histogram bin for the current gradient
            int cellX = idx / cellSize;
            int cellY = idy / cellSize;
            int histIndex = cellY * (width / cellSize) + cellX;
            int numBins = 9; // Assuming 9 orientation bins
            float binWidth = M_PI / numBins;
            int bin = floor((d_orientation + M_PI) / binWidth);
            if (bin == numBins) bin = 0; // Wrap around

            d_histograms[histIndex * numBins + bin] += d_magnitude;
        }
    }
}

std::vector<float> computeDescriptors(const std::string& image_path){
  std::cout << "Start" << std::endl;
    // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }

    // Start measuring time
    auto start = std::chrono::high_resolution_clock::now();

    // Allocate memory for histograms
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;

    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    computeGradientsSequential(image, h_histograms, cellSize);

    auto end = std::chrono::high_resolution_clock::now();

    // Compute the duration and print it
    std::chrono::duration<double> duration = end - start;
    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;

    // Normalization
    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }*/
    std::cout << "CUGAWANGA" << std::endl;
    std::vector<float> descriptor;
    descriptor.push_back(h_histograms[0]);
    //delete[] h_histograms;

    return descriptor;
}


int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    for (int i=1; i <= 144; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();

     std::string image_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0/117.png";

     std::vector<float> descriptor = computeDescriptors(image_path);
     std::cout << "Compute Descriptor" << std::endl;
    if (descriptor.empty()) {
        std::cout << "Vector is empty" << std::endl;
    } else {
        int label = 1;
        std::vector<float> descriptor1;
        int length = static_cast<int>(descriptor.size());
        std::cout << "Dimension of descriptor: " << length << std::endl;
        descriptor1.push_back(length);
        saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
        descriptor.clear(); // Clear the vector
        descriptor1.clear();
    }

    return 0;
}


Overwriting hello.cpp


In [ ]:
!g++ hello.cpp -o hello `pkg-config --cflags --libs opencv4`
!./hello


Start
Execution time: 0.0119241 seconds
CUGAWANGA
malloc(): invalid size (unsorted)


In [ ]:
/*std::vector<float> computeDescriptors(const std::string& image_path){
    // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }

    // Start measuring time
    auto start = std::chrono::high_resolution_clock::now();

    // Allocate memory for histograms
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;

    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    computeGradientsSequential(image, h_histograms, cellSize);

    auto end = std::chrono::high_resolution_clock::now();

    // Compute the duration and print it
    std::chrono::duration<double> duration = end - start;
    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;

    // Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    std::cout << "CUGAWANGA" << std::endl;

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }

    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;
    delete[] h_histograms;
    return descriptor;
}*/

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame with features and labels
# Splitting data into features (X) and labels (y)

df = pd.read_csv("descriptor.csv")
new_value = 0

df.iloc[-1, df.columns.get_loc('label')] = new_value

X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']


In [ ]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,label
0,0.644542,0.270649,0.209346,0.219895,0.243296,0.205410,0.306049,0.317417,0.351183,0.425772,...,0.227495,0.164700,0.247566,0.386196,0.698530,0.371040,0.199949,0.164022,0.134855,1
1,0.489285,0.313330,0.338145,0.378378,0.395283,0.272598,0.267245,0.266543,0.178574,0.235455,...,0.344043,0.331950,0.361637,0.300967,0.361063,0.318854,0.367762,0.309161,0.295327,1
2,0.603244,0.262909,0.288703,0.275478,0.254339,0.257944,0.336061,0.273818,0.297660,0.380624,...,0.430357,0.329545,0.352977,0.268352,0.402616,0.313292,0.241961,0.260699,0.350467,1
3,0.688987,0.268127,0.223381,0.217443,0.241237,0.232688,0.271446,0.276305,0.306363,0.418343,...,0.263961,0.188683,0.210332,0.207616,0.385607,0.262730,0.543682,0.392910,0.373754,1
4,0.575349,0.291004,0.272408,0.250846,0.255007,0.317185,0.290041,0.332377,0.294834,0.293702,...,0.343681,0.257530,0.296350,0.317993,0.333191,0.291995,0.419408,0.368308,0.344657,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.406344,0.115792,0.111428,0.116107,0.839464,0.121537,0.117669,0.129851,0.213078,0.271225,...,0.230496,0.180408,0.135713,0.344164,0.678173,0.508680,0.101348,0.128973,0.178546,0
518,0.410489,0.093417,0.097809,0.255268,0.694336,0.433774,0.179545,0.144973,0.156615,0.357678,...,0.336548,0.128620,0.203286,0.252332,0.420822,0.561934,0.138108,0.127756,0.486764,0
519,0.888679,0.030741,0.044936,0.031906,0.391852,0.044424,0.030935,0.032400,0.220770,0.670775,...,0.577010,0.249210,0.216837,0.263973,0.359025,0.437901,0.263333,0.227103,0.216050,0
520,0.790527,0.038492,0.051095,0.054766,0.370023,0.064761,0.124946,0.107843,0.446791,0.508999,...,0.456539,0.209702,0.103131,0.375936,0.406504,0.181839,0.176525,0.218455,0.564303,0


In [ ]:
y

0    1
1    1
2    1
3    0
Name: label, dtype: int64

In [ ]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Linear Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)

# Making predictions
y_pred = svm.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6857142857142857


In [ ]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>


namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

//__global__ void computeGradients(unsigned char* image, float *d_magnitude, float *d_orientation, float *d_histograms, int width, int height, int cellSize) {
__global__ void computeGradients(unsigned char* image, float *d_histograms, int width, int height, int cellSize) {

    int idx = (blockIdx.x * blockDim.x) + threadIdx.x;
    int idy = (blockIdx.y * blockDim.y) + threadIdx.y;
    //int indexCurrent = idy * width + idx;

    //int blockId = blockIdx.x + blockIdx.y * gridDim.x;
    //int threadId = blockId * (blockDim.x * blockDim.y) + (threadIdx.y * blockDim.x) + threadIdx.x;


    if (idx >= width || idy >= height) return; // Boundary check

    float G_x = 0;
    if (idx > 0 && idx < width - 1) {
        G_x = (float)image[idy * width + (idx + 1)] - (float)image[idy * width + (idx - 1)];
    }

    float G_y = 0;
    if (idy > 0 && idy < height - 1) {
        G_y = (float)image[(idy + 1) * width + idx] - (float)image[(idy - 1) * width + idx];
    }
    float d_magnitude = sqrtf(G_x * G_x + G_y * G_y);
    float d_orientation = atan2f(G_y, G_x);
    //d_magnitude[indexCurrent] = sqrtf(G_x * G_x + G_y * G_y);
    //d_orientation[indexCurrent] = atan2f(G_y, G_x);

    // Compute histogram bin for the current gradient
    int cellX = idx / cellSize;
    int cellY = idy / cellSize;
    int histIndex = cellY * (width / cellSize) + cellX;
    int numBins = 9; // Assuming 9 orientation bins
    float binWidth = M_PI / numBins;
    //int bin = floor((d_orientation[indexCurrent] + M_PI) / binWidth);
    int bin = floor((d_orientation + M_PI) / binWidth);
    if (bin == numBins) bin = 0; // Wrap around
    //atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude[indexCurrent]);
    //atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude);

    /*if((idx == 2 && idy == 2) || (idx==300 & idy == 300)){
      //printf("Value for idx %d e idy %d - bin value: %d - orientation value %f \n", idx, idy, bin,d_orientation[indexCurrent] + M_PI);
    }*/
}

std::vector<float> computeDescriptors(const std::string& image_path){
    // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(224, 224)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }



    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);
    cudaError_t status;
    /*size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    status = cudaMalloc((void **)&d_magnitude, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_magnitude, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    sizeInBytes = image.total() * sizeof(float);
    float* d_orientation;
    status = cudaMalloc((void **)&d_orientation, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_orientation, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }*/

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(8, 8);
    dim3 gridSize((image.cols) / blockSize.x,
                  (image.rows) / blockSize.y);


    // Allocate memory for histograms
    int cellSize = 64;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    size_t histSize = numCellsX * numCellsY * 9 * sizeof(float);
    float* d_histograms;
    status = cudaMalloc((void **)&d_histograms, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_histograms, 0, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    // Start measuring time
    auto start = std::chrono::high_resolution_clock::now();
    // Launch the kernel
    //computeGradients<<<gridSize, blockSize>>>(d_image, d_magnitude, d_orientation, d_histograms, image.cols, image.rows, cellSize);
    computeGradients<<<gridSize, blockSize>>>(d_image, d_histograms, image.cols, image.rows, cellSize);

    cudaDeviceSynchronize();

    auto end = std::chrono::high_resolution_clock::now();

    // Compute the duration and print it
    std::chrono::duration<double> duration = end - start;
    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;


    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    cudaMemcpy(h_histograms, d_histograms, histSize, cudaMemcpyDeviceToHost);

    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        for (int j = 0; j < 9; ++j) {
            std::cout << "Cell " << i << ", Bin " << j << ": " << h_histograms[i * 9 + j] << std::endl;
        }
    }*/
    // Cleanup

    // Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }
    cudaFree(d_image);
    //cudaFree(d_magnitude);
    //cudaFree(d_orientation);
    delete[] h_histograms;
    cudaFree(d_histograms);

    return descriptor;
}

int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    for (int i=1; i <= 144; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();

    //People present class
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;


        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 1;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }
    }


    return 0;
}


In [10]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>
#include <chrono>
#include <fstream>
#include <sstream>
#include <filesystem>
#include <chrono>


namespace fs = std::filesystem;

int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1";
    std::vector<cv::Mat> images;

    // Load each image from file paths
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        cv::Mat image = cv::imread(file_path);
        if (image.empty()) {
            std::cerr << "Failed to load image: " << file_path << std::endl;
            continue;
        }
        std::cout << "Load cv2 " << file_path << std::endl;
        images.push_back(image);
    }

    std::cout << "Size of images vector: " << images.size() << std::endl;

    // Check if any images were loaded
    if (images.empty()) {
        std::cerr << "No images loaded." << std::endl;
        return 1;
    }

    // Start measuring time
    cudaError_t status;
    auto start = std::chrono::high_resolution_clock::now();
    // Allocate GPU memory for images
    std::vector<uchar*> d_images(images.size());
    std::vector<size_t> image_sizes(images.size());
    for (size_t i = 0; i < images.size(); ++i) {
        image_sizes[i] = images[i].total() * images[i].elemSize();
        status = cudaMalloc(&d_images[i], image_sizes[i]);
        if (status != cudaSuccess) {
            // Handle error (e.g., printing an error message and exiting)
            fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
            exit(EXIT_FAILURE);
        }
        status = cudaMemcpy(d_images[i], images[i].data, image_sizes[i], cudaMemcpyHostToDevice);
        if (status != cudaSuccess) {
            // Handle error (e.g., printing an error message and exiting)
            fprintf(stderr, "cudaMemcpy failed: %s\n", cudaGetErrorString(status));
            exit(EXIT_FAILURE);
        }
    }
    auto end = std::chrono::high_resolution_clock::now();

    // Compute the duration and print it
    std::chrono::duration<double> duration = end - start;
    std::cout << "Execution time: " << duration.count() << " seconds" << std::endl;

    // Free GPU memory
    for (size_t i = 0; i < images.size(); ++i) {
        status = cudaFree(d_images[i]);
        if (status != cudaSuccess) {
            // Handle error (e.g., printing an error message and exiting)
            fprintf(stderr, "CudaFree failed: %s\n", cudaGetErrorString(status));
            exit(EXIT_FAILURE);
        }
    }


    return 0;
}


Overwriting gradient_computation.cu


In [11]:
!nvcc gradient_computation.cu -o gradient_computation -diag-suppress 611 `pkg-config --cflags --libs opencv4` -run


Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/10.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/108.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/117.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/100.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/1.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/113.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/111.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/11.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/121.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/104.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/118.png
Load cv2 /content/drive/My Drive/GPU Computing/human detection dataset/1/112.png
Load cv2 /content/drive/My Drive